In [10]:
from bs4 import BeautifulSoup
import re
import string
import google
from google.cloud import storage
import os
from io import StringIO
from datetime import datetime
import json, session_info, requests
import pandas as pd 
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.api_core.datetime_helpers import DatetimeWithNanoseconds

In [14]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials .from_json_keyfile_name(r"E:\Downloads\Google Cloud Project\Credentials.json", scope)
client = gspread.authorize(creds)
sheet = client.open('Econ 446 Final').sheet1

In [20]:
url = "https://www.baseball-reference.com/teams/PHI/2024.shtml"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'id': 'team_batting'})

# Read the table into a DataFrame
df = pd.read_html(str(table))[0]
df = df.replace({float('inf'): None, float('-inf'): None})
df = df.fillna('')

spreadsheet_name = 'Phillies Batting Data'
spreadsheet = client.open(spreadsheet_name)
sheet = spreadsheet.sheet1  
sheet.clear()
sheet.update([df.columns.values.tolist()] + df.values.tolist())

print(f"Data successfully written to Google Sheet: {spreadsheet_name}")

Data successfully written to Google Sheet: Phillies Batting Data


In [25]:
# Scraping
url = "https://www.baseball-reference.com/teams/PHI/2024.shtml"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find and read Data Frame
table = soup.find('table', {'id': 'team_batting'})
if table is None:
    raise ValueError("Could not find the batting table.")
df = pd.read_html(str(table))[0]
print("DataFrame loaded:")
print(df.head())

# Remove NANS
df = df.replace({float('inf'): None, float('-inf'): None})
df = df.fillna('')

# Use APIs needed
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(r"E:\Downloads\Google Cloud Project\Credentials.json", scopes=scope)
client = gspread.authorize(creds)

# Locate and Update Sheet as needed
spreadsheet_name = 'Phillies Batting Data'
spreadsheet = client.open(spreadsheet_name)
sheet = spreadsheet.sheet1  
sheet.clear()

# Finish
try:
    sheet.update([df.columns.values.tolist()] + df.values.tolist())
    print(f"Data successfully written to Google Sheet: {spreadsheet_name}")
except Exception as e:
    print(f"An error occurred: {e}")

DataFrame loaded:
  Rk Pos                     Name Age   G   PA   AB   R   H  2B  ...   OBP  \
0  1   C            J.T. Realmuto  33  51  223  207  28  54   8  ...  .309   
1  2  1B            Bryce Harper*  31  60  263  220  36  61  11  ...  .388   
2  3  2B            Bryson Stott*  26  60  231  194  32  47   6  ...  .342   
3  4  SS  Trea Turner (10-day IL)  31  33  148  137  27  47  10  ...  .392   
4  5  3B                Alec Bohm  27  64  270  244  30  71  23  ...  .348   

    SLG   OPS OPS+   TB GDP HBP SH SF IBB  
0  .411  .720  105   85   3   3  0  1   0  
1  .532  .920  162  117   7   0  0  2   7  
2  .371  .713  105   72   5   3  0  5   1  
3  .460  .852  144   63   2   2  0  0   0  
4  .467  .815  132  114   8   3  0  3   1  

[5 rows x 28 columns]
Data successfully written to Google Sheet: Phillies Batting Data


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import gspread
from google.oauth2.service_account import Credentials

def batting_sheet(request):
    try:
        # Scrape 
        url = "https://www.baseball-reference.com/teams/PHI/2024.shtml"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'id': 'team_batting'})
        if table is None:
            return "Could not find the batting table.", 400

        # Read table in as DataFrame and remove NaNs and other non Json compliant thigngs
        df = pd.read_html(str(table))[0]
        df = df.replace({float('inf'): None, float('-inf'): None})
        df = df.fillna('')

        # Authenticate with Google Sheets API
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
                 "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
        creds = Credentials.from_service_account_file('credentials.json', scopes=scope)
        client = gspread.authorize(creds)

        # Open existing sheet and clear for new data 
        spreadsheet_name = 'Phillies Batting Data'
        spreadsheet = client.open(spreadsheet_name)
        sheet = spreadsheet.sheet1 
        sheet.clear()
        sheet.update([df.columns.values.tolist()] + df.values.tolist())

        return f"Data successfully written to Google Sheet: {spreadsheet_name}", 200
    except Exception as e:
        return f"An error occurred: {e}", 500